In [2]:
from mesa.batchrunner import BatchRunner, BatchRunnerMP
from mesa.time import RandomActivation
from model import *
from customizeTime import *
import pickle
import numpy as np
import uuid

from customizeTime import *
import env_settings

#M_1 model part
TYPE_MAP = {0:'vert',
            1:'horz'}
settings.MUTATION_PRO = 0.01



class A_2(GenAgent):
    def muta_genetic_info(self):
        # no need for mutation
        pass
    
        
class M_2(GenModel):
    def get_agent_config(self):
        return A_2
    
    def init_env(self):
        #no influence
        env_vals = [self.env_val] * 8000
        return env_vals
    
    def get_schedule(self):
        return RandomActivationMutationWithConstrain
    #
    def __init__(self, N, width, height, specific_kargs):
        #mut_var, init_var, env_val
        init_ratio = specific_kargs['init_ratio']
        self.env_val = specific_kargs['env_val']
        
        
        settings.INIT_MEAN = specific_kargs['gene_mean']
        settings.INIT_SD = specific_kargs['gene_sd']
        settings.INIT_SD_LOW = -1 * 2 * settings.INIT_SD
        settings.INIT_SD_HIGH = 1 * 2 * settings.INIT_SD

        settings.POPU_MUTA_SD = specific_kargs['muta_sd']
        settings.POPU_MUTA_LOW = -2 * settings.POPU_MUTA_SD
        settings.POPU_MUTA_HIGH = 2 * settings.POPU_MUTA_SD
        
        self.specific_kargs_str = '_'.join([str(i) for i in (init_ratio, settings.INIT_SD ,settings.POPU_MUTA_SD)])
        super().__init__(N, width, height, init_ratio)
        

    
    def model_level_record_init(self):
        data_file_name = "./M2_" + self.specific_kargs_str + '_' + str(uuid.uuid1())
        print('data in file ', data_file_name)
        self.data_file = open(data_file_name, 'w')
        self.data_file.write('#' + '\t'.join((TYPE_MAP[0], TYPE_MAP[0] + ' beta', TYPE_MAP[0] + 'gene', TYPE_MAP[1], TYPE_MAP[1] + ' beta', TYPE_MAP[1] + 'gene'))+ '\n')
       
    def get_popu_beta(self, gen_type):
        return np.mean([agent.self_gen_cof for agent in self.schedule.agents if agent.gen_type == gen_type])
    
    def get_popu_gene_mean(self, gen_type):
        return np.mean([np.mean(agent.gen_info) for agent in self.schedule.agents if agent.gen_type == gen_type])
    
    def collect_data(self):
        return (self.get_ver_popu_size(), self.get_popu_beta(0), self.get_popu_gene_mean(0),
                self.get_her_popu_size(), self.get_popu_beta(1), self.get_popu_gene_mean(1))
        
        
    def step(self):
        data = self.collect_data()
        print(data)
        self.data_file.write('\t'.join([str(d) for d in data]) + '\n')
        super().step()
        
        


In [2]:
fixed_params = {'N':2000, 'width':100, 'height':100}

variable_params = {'specific_kargs': [{'env_val': 0.5 , 'init_ratio':0.5, 'gene_mean':0.5, 'gene_sd':0.1, 'muta_sd':0.01}]}

batch_run = BatchRunnerMP(
    M_2, 5,
    variable_parameters=variable_params,
    fixed_parameters=fixed_params,
    iterations=10,
    max_steps=800,
)
batch_run.run_all()

0it [00:00, ?it/s]

data in file  ./M2_0.5_0.1_0.1_70f083aa-bfd9-11e9-9525-f2189850a77c
data in file  ./M2_0.5_0.1_0.1_70f0d922-bfd9-11e9-8fd6-f2189850a77c
data in file  ./M2_0.5_0.1_0.1_70f0e016-bfd9-11e9-86ee-f2189850a77c
data in file  ./M2_0.5_0.1_0.1_70f1be8c-bfd9-11e9-a837-f2189850a77c
data in file  ./M2_0.5_0.1_0.1_70f21b2a-bfd9-11e9-be84-f2189850a77c
(970, 0.0, 1030, 0.0)
0 population size:  2000
(970, 0.0, 1030, 0.0)
0 population size:  2000
(970, 0.0, 1030, 0.0)
0 population size:  2000
(970, 0.0, 1030, 0.0)
(970, 0.0, 1030, 0.0)
0 population size:  2000
0 population size:  2000
(688, 0.22985988372093022, 779, 0.23225070089858788)
1 population size:  1467
(695, 0.22916468489208633, 785, 0.23393096560509555)
1 population size:  1480
(715, 0.22460267412587412, 741, 0.23452750337381914)
1 population size:  1456
(726, 0.23364683195592284, 742, 0.23107676549865228)
1 population size:  1468
(720, 0.24023542222222222, 769, 0.21741297269180754)
1 population size:  1489
(508, 0.21191501873089533, 601, 0.2

20 population size:  7399
(7817, 0.7323201076595385, 148, 0.5321325692772412)
20 population size:  7965
(7498, 0.710515055048993, 261, 0.5294454692469535)
21 population size:  7759
(7329, 0.7140100623509956, 344, 0.5353836075229842)
21 population size:  7673
(7617, 0.7168745602008623, 251, 0.535255068701625)
21 population size:  7868
(7765, 0.7351889199216313, 248, 0.5653249958370702)
21 population size:  8013
(8329, 0.7832030252723573, 107, 0.5440299448981266)
21 population size:  8436
(7997, 0.757501837426907, 225, 0.5352869692687372)
22 population size:  8222
(7819, 0.7478519297762977, 289, 0.5832169173170255)
22 population size:  8108
(8035, 0.7672723583384066, 194, 0.5980825813272458)
22 population size:  8229
(8198, 0.7749751359868157, 216, 0.586032602359848)
22 population size:  8414
(8686, 0.8133065508373144, 79, 0.5853412127164926)
22 population size:  8765
(8397, 0.7890078724657729, 185, 0.5945643264491767)
23 population size:  8582
(8260, 0.7758415993442555, 222, 0.596516328

42 population size:  9816
(9795, 0.9639339399636064, 1, 0.8651248328012773)
41 population size:  9796
(9801, 0.9646516489215585, 1, 0.768424385361657)
42 population size:  9802
(9798, 0.9604736717683242, 10, 0.8624904153923885)
42 population size:  9808
(9810, 0.9650512770356664, 7, 0.8779609147201903)
43 population size:  9817
(9814, 0.965278248908709, 12, 0.8883989420695922)
43 population size:  9826
(9807, 0.9665724912205929, 1, 0.8651248328012773)
42 population size:  9808
(9824, 0.9655270103709848, 1, 0.768424385361657)
43 population size:  9825
(9794, 0.9650712898837529, 9, 0.8701775637059157)
43 population size:  9803
(9835, 0.9678760797174201, 7, 0.8836726908512814)
44 population size:  9842
(9846, 0.9668188959576755, 1, 0.8792874450237647)
43 population size:  9847
(9834, 0.9652197584979816, 12, 0.8898248606745028)
44 population size:  9846
(9840, 0.9686312631342057, 1, 0.768424385361657)
44 population size:  9841
(9804, 0.9667046737065131, 9, 0.8746451289789002)
44 population

/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

413 population size:  9554
(9566, 0.9180717515120279, 0, nan)
414 population size:  9566
(9491, 0.9155568373090344, 101, 0.898357618110424)
413 population size:  9592
(9559, 0.9167451854001043, 0, nan)
413 population size:  9559
(9549, 0.9213055563506404, 0, nan)
413 population size:  9549
(9561, 0.9184682077065015, 2, 0.9260628747237614)
414 population size:  9563
(9537, 0.9157818419701546, 0, nan)
415 population size:  9537
(9472, 0.9219842549838422, 100, 0.9089956262811905)
414 population size:  9572
(9549, 0.9170715685611244, 0, nan)
414 population size:  9549
(9556, 0.9212175611657399, 0, nan)
414 population size:  9556
(9549, 0.9180657132468202, 2, 0.9260628747237614)
415 population size:  9551
(9559, 0.9165539678766309, 0, nan)
416 population size:  9559
(9473, 0.9213828101240286, 96, 0.9041554114489884)
415 population size:  9569
(9550, 0.915971570326864, 0, nan)
415 population size:  9550
(9545, 0.918244717467782, 0, nan)
415 population size:  9545
(9585, 0.9173813954660431, 2

(9560, 0.918652521016326, 0, nan)
438 population size:  9560
(9477, 0.9186572145810744, 62, 0.9291697092250368)
437 population size:  9539
(9535, 0.9172905980730062, 0, nan)
437 population size:  9535
(9573, 0.9206035648675815, 0, nan)
437 population size:  9573
(9563, 0.917493303770809, 3, 0.9061531822725019)
438 population size:  9566
(9591, 0.9185261007070104, 0, nan)
439 population size:  9591
(9484, 0.919139817803563, 61, 0.9126094409026586)
438 population size:  9545
(9532, 0.9128322812306994, 0, nan)
438 population size:  9532
(9574, 0.9208399794407818, 0, nan)
438 population size:  9574
(9552, 0.9191822254619186, 2, 0.8812516796200595)
439 population size:  9554
(9571, 0.9201927764192328, 0, nan)
440 population size:  9571
(9491, 0.9192266380930711, 59, 0.9095134175770606)
439 population size:  9550
(9522, 0.9154041551155567, 0, nan)
439 population size:  9522
(9556, 0.9219869412211534, 0, nan)
439 population size:  9556


/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(9513, 0.9191453862774778, 0, nan)
440 population size:  9513
(9560, 0.9177339415059873, 0, nan)
441 population size:  9560
(9523, 0.9184150041361311, 58, 0.9260549092614913)
440 population size:  9581
(9565, 0.9145335721223232, 0, nan)
440 population size:  9565
(9569, 0.9233011798102384, 0, nan)
440 population size:  9569
(9528, 0.9194220869256604, 0, nan)
441 population size:  9528
(9556, 0.9203218610871314, 0, nan)
442 population size:  9556
(9506, 0.9213281205701734, 58, 0.8758131591334799)
441 population size:  9564
(9528, 0.9159181475923056, 0, nan)
441 population size:  9528
(9571, 0.9183933003017587, 0, nan)
441 population size:  9571
(9562, 0.9186001192816727, 0, nan)
442 population size:  9562
(9520, 0.9196647930941342, 0, nan)
443 population size:  9520
(9537, 0.9175934138189185, 0, nan)
442 population size:  9537
(9515, 0.9211765731970303, 57, 0.8936777311347925)
442 population size:  9572
(9550, 0.9206926656721888, 0, nan)
442 population size:  9550
(9592, 0.9166342882120

(9527, 0.9177432902305769, 0, nan)
466 population size:  9527
(9483, 0.9199277752471573, 34, 0.8457463495218924)
465 population size:  9517
(9561, 0.9189418938163821, 0, nan)
465 population size:  9561
(9534, 0.9205937563876305, 0, nan)
465 population size:  9534
(9574, 0.9202285339803136, 0, nan)
466 population size:  9574
(9558, 0.9165729923631476, 0, nan)
467 population size:  9558
(9498, 0.9176309706526227, 33, 0.9615494856908663)
466 population size:  9531
(9564, 0.9189405835164676, 0, nan)
466 population size:  9564
(9557, 0.9186938926103019, 0, nan)
466 population size:  9557
(9558, 0.9190934625162215, 0, nan)
467 population size:  9558
(9548, 0.9176808796063194, 0, nan)
468 population size:  9548
(9500, 0.9163472481977198, 35, 0.8799014663013482)
467 population size:  9535
(9583, 0.9192736701970798, 0, nan)
467 population size:  9583
(9592, 0.916761773253573, 0, nan)
467 population size:  9592
(9569, 0.9181421972339451, 0, nan)
468 population size:  9569
(9537, 0.92236208161165

In [2]:
fixed_params = {'N':2000, 'width':100, 'height':100}

variable_params = {'specific_kargs': [{'env_val': 0.25 , 'init_ratio':0.5, 'gene_mean':0.5, 'gene_sd':0.1, 'muta_sd':0.005}]}

batch_run = BatchRunnerMP(
    M_2, 5,
    variable_parameters=variable_params,
    fixed_parameters=fixed_params,
    iterations=10,
    max_steps=800,
)
batch_run.run_all()

0it [00:00, ?it/s]

data in file  ./M2_0.5_0.1_0.005_15129a9c-c131-11e9-bbb1-f2189850a77c
data in file  ./M2_0.5_0.1_0.005_15132af2-c131-11e9-bb59-f2189850a77c
data in file  ./M2_0.5_0.1_0.005_15143924-c131-11e9-b400-f2189850a77c
data in file  ./M2_0.5_0.1_0.005_1514ab3e-c131-11e9-bc8f-f2189850a77c
data in file  ./M2_0.5_0.1_0.005_151591f2-c131-11e9-a571-f2189850a77c
(1002, 0.0, 998, 0.0)
(1002, 0.0, 998, 0.0)
0 population size:  2000
0 population size:  2000
(1002, 0.0, 998, 0.0)
0 population size:  2000
(1002, 0.0, 998, 0.0)
(1002, 0.0, 998, 0.0)
0 population size:  2000
0 population size:  2000
(1275, 0.4139784062745099, 1269, 0.41502465563435775)
1 population size:  2544
(1315, 0.40403545247148287, 1259, 0.4224970325655282)
1 population size:  2574
(1301, 0.41065586164488854, 1255, 0.41287672350597615)
1 population size:  2556
(1281, 0.4234089586260734, 1300, 0.40361208307692303)
1 population size:  2581
(1291, 0.4133643841982959, 1312, 0.41997298475609757)
1 population size:  2603
(1639, 0.4295358657

(5706, 0.7922815581455098, 2938, 0.7382923718892099)
20 population size:  8644
(5721, 0.79357627589573, 2874, 0.7367889468434129)
20 population size:  8595
(5704, 0.7949639725325768, 2936, 0.7466660679800031)
20 population size:  8640
(5999, 0.8017496455588023, 2690, 0.7603399297421901)
21 population size:  8689
(6126, 0.8154850617336689, 2641, 0.7552909217483733)
21 population size:  8767
(5914, 0.8111149397635705, 2832, 0.7542948184561151)
21 population size:  8746
(5882, 0.8134375379932954, 2805, 0.7539963797766577)
21 population size:  8687
(5918, 0.8093546401647088, 2806, 0.7640318169469488)
21 population size:  8724
(6254, 0.8158469321328038, 2562, 0.7672959321723544)
22 population size:  8816
(6332, 0.8281234026946868, 2510, 0.7735054447328449)
22 population size:  8842
(6101, 0.821062085793629, 2758, 0.7593237853358434)
22 population size:  8859
(6065, 0.8221602546240604, 2727, 0.7602486662459143)
22 population size:  8792
(6130, 0.8245909061396485, 2703, 0.7732786943507417)
22

41 population size:  9764
(8317, 0.9569344927400352, 1389, 0.915436737939181)
41 population size:  9706
(8430, 0.9548948358657103, 1331, 0.9167051049917194)
41 population size:  9761
(8454, 0.9559962549131458, 1306, 0.9197926640005447)
41 population size:  9760
(8637, 0.957798657150437, 1142, 0.9186942523410528)
42 population size:  9779
(8666, 0.9613838210265555, 1118, 0.9254644218049828)
42 population size:  9784
(8390, 0.958840312289264, 1355, 0.9140309431277989)
42 population size:  9745
(8470, 0.9603641195652725, 1292, 0.9306072329232975)
42 population size:  9762
(8506, 0.960267845648918, 1262, 0.9279341082636083)
42 population size:  9768
(8685, 0.9610308581005119, 1111, 0.9263441734309337)
43 population size:  9796
(8683, 0.9634092628167774, 1077, 0.9341411723535821)
43 population size:  9760
(8456, 0.960097723252154, 1324, 0.922294466107063)
43 population size:  9780
(8516, 0.9608347900993531, 1258, 0.9312991862563076)
43 population size:  9774
(8569, 0.9603477676986918, 1213,

62 population size:  9879
(9031, 0.9743120237406745, 817, 0.97219408608133)
62 population size:  9848
(8901, 0.9759008088355107, 964, 0.9671791109153391)
62 population size:  9865
(8853, 0.9754220452724627, 1017, 0.9771530863476501)
62 population size:  9870
(9111, 0.9745489941351083, 770, 0.9747559001223662)
63 population size:  9881
(8823, 0.9750948765128704, 1031, 0.9814600710091133)
63 population size:  9854
(9016, 0.9740584545143864, 833, 0.9592922773455084)
63 population size:  9849
(8912, 0.9741678097570842, 966, 0.9712828976804213)
63 population size:  9878
(8857, 0.9769362568649281, 1016, 0.9744524167618916)
63 population size:  9873
(9120, 0.9766742191423113, 769, 0.9807566686390357)
64 population size:  9889
(9023, 0.9742798740625377, 834, 0.9730998931912669)
64 population size:  9857
(8826, 0.9732664701750935, 1036, 0.9747563586337197)
64 population size:  9862
(8916, 0.9750251658343505, 961, 0.9783392626387499)
64 population size:  9877
(8860, 0.9759233174484304, 1018, 0.9

...

data in file  ./M2_0.5_0.1_0.005_3f9a455a-c134-11e9-81fd-f2189850a77c
data in file  ./M2_0.5_0.1_0.005_3f9ad31c-c134-11e9-a4be-f2189850a77c
data in file  ./M2_0.5_0.1_0.005_3f9adaf6-c134-11e9-8cb6-f2189850a77c
data in file  ./M2_0.5_0.1_0.005_3f9b95ea-c134-11e9-9722-f2189850a77c
data in file  ./M2_0.5_0.1_0.005_3f9cf43a-c134-11e9-bd5f-f2189850a77c


KeyboardInterrupt: 

(1002, 0.0, 998, 0.0)
0 population size:  2000
(1002, 0.0, 998, 0.0)
0 population size:  2000
(1002, 0.0, 998, 0.0)
0 population size:  2000
(1002, 0.0, 998, 0.0)
0 population size:  2000
(1002, 0.0, 998, 0.0)
0 population size:  2000


Process ForkPoolWorker-10:
Process ForkPoolWorker-6:
Process ForkPoolWorker-9:
Process ForkPoolWorker-7:
Process ForkPoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)

  File "/usr/local/lib/python3.7/site-packages/mesa/space.py", line 256, in out_of_bounds
    def out_of_bounds(self, pos):
  File "/usr/local/lib/python3.7/site-packages/mesa/space.py", line 455, in <genexpr>
    self[x][y] for x, y in cell_list if not self.is_cell_empty((x, y)))
  File "/usr/local/lib/python3.7/site-packages/mesa/space.py", line 160, in iter_neighborhood
    if not self.torus and (not (0 <= dx + x < self.width) or not (0 <= dy + y < self.height)):
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt


In [3]:
fixed_params = {'N':2000, 'width':100, 'height':100}

variable_params = {'specific_kargs': [{'env_val': 0.25 , 'init_ratio':0, 'gene_mean':0.5, 'gene_sd':0.1, 'muta_sd':0.005}]}

batch_run = BatchRunnerMP(
    M_2, 4,
    variable_parameters=variable_params,
    fixed_parameters=fixed_params,
    iterations=10,
    max_steps=800,
)
batch_run.run_all()

0it [00:00, ?it/s]

data in file  ./M2_0_0.1_0.005_c9bd3666-c134-11e9-bb73-f2189850a77c
data in file  ./M2_0_0.1_0.005_c9beee70-c134-11e9-a96a-f2189850a77c
data in file  ./M2_0_0.1_0.005_c9c0b41c-c134-11e9-b433-f2189850a77c
data in file  ./M2_0_0.1_0.005_c9c140f8-c134-11e9-a4bb-f2189850a77c


/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwa

(0, nan, 2000, 0.0)
(0, nan, 2000, 0.0)
(0, nan, 2000, 0.0)
0 population size:  2000


/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

0 population size:  2000
(0, nan, 2000, 0.0)


/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 population size:  2000
(0, nan, 2000, 0.0)
0 population size:  2000
(0, nan, 2550, 0.4165509992156863)
1 population size:  2550
(0, nan, 2581, 0.4140226625339016)
1 population size:  2581
(0, nan, 2588, 0.4045016800618238)
1 population size:  2588
(0, nan, 2603, 0.41236873761044956)
1 population size:  2603
(0, nan, 3216, 0.4297246061162109)
2 population size:  3216
(0, nan, 3222, 0.43435888457691274)
2 population size:  3222
(0, nan, 3210, 0.4368225696380451)
2 population size:  3210
(0, nan, 3267, 0.43513705190558905)
2 population size:  3267
(0, nan, 3893, 0.4518753434738371)
3 population size:  3893
(0, nan, 3867, 0.44429132042641806)
3 population size:  3867
(0, nan, 3921, 0.4574752289933644)
3 population size:  3921
(0, nan, 3942, 0.4535729989043908)
3 population size:  3942
(0, nan, 4536, 0.47609099134838007)
4 population size:  4536
(0, nan, 4621, 0.4838426142184196)
4 population size:  4621
(0, nan, 4543, 0.47039291292060426)
4 population size:  4543
(0, nan, 4631, 0.4732972

(0, nan, 9105, 0.8402685961343828)
34 population size:  9105
(0, nan, 9033, 0.8365313920203521)
34 population size:  9033
(0, nan, 9077, 0.84563124516346)
35 population size:  9077
(0, nan, 9080, 0.8514358857788037)
35 population size:  9080
(0, nan, 9146, 0.8494491909789014)
35 population size:  9146
(0, nan, 9115, 0.8433333420506496)
35 population size:  9115
(0, nan, 9159, 0.8453352610311908)
36 population size:  9159
(0, nan, 9128, 0.8542991788175086)
36 population size:  9128
(0, nan, 9162, 0.8574104868860778)
36 population size:  9162
(0, nan, 9138, 0.853744968277444)
36 population size:  9138
(0, nan, 9219, 0.8568390233412546)
37 population size:  9219
(0, nan, 9175, 0.8581630519933563)
37 population size:  9175
(0, nan, 9209, 0.8613157376507053)
37 population size:  9209
(0, nan, 9177, 0.857241382978544)
37 population size:  9177
(0, nan, 9244, 0.8688463726073634)
38 population size:  9244
(0, nan, 9269, 0.8629527666618555)
38 population size:  9269
(0, nan, 9201, 0.86951977486

68 population size:  9904
(0, nan, 9905, 0.9831158261152103)
68 population size:  9905
(0, nan, 9910, 0.9826824087402517)
68 population size:  9910
(0, nan, 9909, 0.9825901960689719)
68 population size:  9909
(0, nan, 9914, 0.9823865610420578)
69 population size:  9914
(0, nan, 9915, 0.9825024083790999)
69 population size:  9915
(0, nan, 9920, 0.9838090353233492)
69 population size:  9920
(0, nan, 9916, 0.9844032829022683)
69 population size:  9916
(0, nan, 9917, 0.9841395895122257)
70 population size:  9917
(0, nan, 9938, 0.984015952196546)
70 population size:  9938
(0, nan, 9928, 0.9850201384803825)
70 population size:  9928
(0, nan, 9932, 0.9847660786838229)
70 population size:  9932
(0, nan, 9929, 0.9858026442894885)
71 population size:  9929
(0, nan, 9939, 0.9866280667477877)
71 population size:  9939
(0, nan, 9927, 0.9875201873587875)
71 population size:  9927
(0, nan, 9919, 0.9858178504084055)
71 population size:  9919
(0, nan, 9934, 0.9867099445541547)
72 population size:  9934